### Анализ текстов джазовых песен

**Описание**:

В качестве материала для анализа мы решили взять песни четырех "классических" исполнителей музыки в жанре "джаз": Фрэнка Синатры, Дина Мартина, Нэта Кинг Коула и Луи Армстронга. Мы взяли по пятьдесят наиболее знаменитых и популярных песен каждого из композиторов (в случае Нэта Кинг Коула вышло только шестнадцать) для анализа по таким параметрам, как продолжительность и наиболее часто используемые слова.

В качестве инструментов для парсинга текста был использован модуль lyricsgenius, продолжительность вытащена с Youtube.

In [ ]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_sm
import json
import lyricsgenius
import re
import time
import spacy
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from lyricsgenius import Genius
from spacy.lang.en.stop_words import STOP_WORDS
from collections import defaultdict, Counter
from nltk import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

# Загрузка данных

Здесь представлено два варианта загрузки данных: 
* Облегченный - рекомендуемый метод, в котором мы используем предварительно сгенерированную системой и сохраненную на системе базу данных.
* Оригинальный - скомпилированный воедино код из трех основных скриптов, скачивающий песни с Genius, собирающий их длительность с YouTube и год выпуска с Discogs. Время от времени api может выдавать ошибку на стороне сервера. Метод оставлен для демонстрации изначальной работы кода для генерации датабазы, однако наш анализ ведется на основе полной базы данных, собранной в файле.

Выберите один из двух типов, запустите его ячейку, и продолжайте ниже.

In [ ]:
# Облегченный метод, позволяющий пропустить ошибки api при генерации очищенного текста. Скачивает final_lyrics,
#успешно прогнанный через все скрипты без ошибок.
df = pd.read_csv('data/final_lyrics')
#начальная предобработка
df['lyrics_clean'] = df['lyrics'].replace(r'[^\w\s]',' ',regex=True).replace(r'\s+',' ',regex=True).str.lower()

In [ ]:
# Оригинальный код через скрипт. Время от времени api выдает ошибку.

def clean_genius_lyrics(text):
    # Удаляем все технические блоки перед текстом песни
    text = re.sub(
        r'^(\d+\s*Contributors|.*?Lyrics\b).*?\n',
        '',
        text,
        flags=re.IGNORECASE | re.DOTALL | re.MULTILINE
    )

    # Удаляем Read More и всё до него
    text = re.split(r'Read More\s*\n', text, flags=re.IGNORECASE)[-1] #Оставляем часть после "Read More"

    # Дополнительные паттерны
    patterns = [
        r'Genius Annotation.*',
        r'You might also like.*',
        r'Embed\s*\d+',
        r'\xa0',
        r'^[\W\d_]+$'  # любые не-буквенные символы, цифры или подчеркивания.
    ]

    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.DOTALL)

    # Финальная очистка
    text = "\n".join([line.strip() for line in text.split("\n") if line.strip()]) # удаляем пустые строки и лишние пробелы и соединяем полученные строки
    return text.strip()
    

load_dotenv()

genius = lyricsgenius.Genius(os.getenv('GENIUS_TOKEN'))
genius.remove_section_headers = True # удаляем секции вида "[Chorus], [Verse n]" и похожего типа
artists = ['Frank Sinatra', 'Dean Martin', 'Nat "King" Cole', 'Louis Armstrong']

all_songs = []

for artist in artists:
    try:
        if artist != 'Nat "King" Cole':
            artist_obj = genius.search_artist(artist, max_songs = 50, sort='popularity')
            for song in artist_obj.songs:
                all_songs.append({
                    'artist': song.artist,
                    'title': song.title,
                    'lyrics': clean_genius_lyrics(song.lyrics) # очищаем текст песни от лишних символов
            })
        else:
            artist_obj = genius.search_artist(artist, max_songs=16, sort='popularity')
            for song in artist_obj.songs:
                all_songs.append({
                    'artist': song.artist,
                    'title': song.title,
                    'lyrics': clean_genius_lyrics(song.lyrics) # очищаем текст песни от лишних символов
                })
    except:
        print(f'Ошибка для {artist}')

lyrics = pd.DataFrame(all_songs)
lyrics = lyrics.drop_duplicates(subset=['artist', 'title', 'lyrics'])  # удаляем возможные дубликаты
lyrics_csv = lyrics.to_csv('lyrics_all.csv',sep=',', index= False)

# Получаем продолжительность

with open(".venv/config.json") as f:
    config = json.load(f)

lyrics = pd.read_csv('lyrics_all.csv')
def get_youtube_duration(artist, title):
    try:
        results = YoutubeSearch(f"{artist} {title}", max_results=1).to_dict()
        return results[0]["duration"]
    except:
        return None

lyrics["duration"] = lyrics.apply(lambda row: get_youtube_duration(row["artist"], row["title"]), axis=1)
def convert_duration(duration):
    minutes = int(duration)
    seconds = round((duration - minutes) * 100)  # Извлекаем две цифры после точки
    return (minutes * 60)  + seconds

# Применяем функцию к колонке duration
lyrics["duration"] = lyrics["duration"].apply(convert_duration)

# Сохраняем результат в новый файл
lyrics.to_csv("lyr_dur1_sec.csv", index=False)

#Загрузим необходимый токен для доступа к сайту

DISCOGS_TOKEN = os.getenv('DISCOGS_TOKEN')

#Получим год релиза песни
def get_discogs_year(artist, title):
    url = f"https://api.discogs.com/database/search?q={artist}+{title}&type=release"
    headers = {"Authorization": f"Discogs token={DISCOGS_TOKEN}"}

    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        return data["results"][0]["year"] if data["results"] else None
    except:
        return None

lyrics = pd.read_csv("data/lyr_dur1_sec.csv")
lyrics["year"] = lyrics.apply(lambda row: get_discogs_year(row["artist"], row["title"]), axis=1)

final_lyrics = lyrics.to_csv('final_lyrics', index=False)

## Очистка данных и подготовка к анализу

В данном разделе мы проведем исключительно разведочный анализ данных (EDA), поскольку данные были предварительно очищены в ходе изначальной генерации датасета в коде выше, и достать чисто этот функционал для раздела не представляется возможным.

Начнем с рассмотрения самого датасета и выясним, из каких колонок он состоит.

In [ ]:
df.head()

Здесь мы можем пронаблюдать следующие колонки:

* **artist** - имя артиста;
* **title** - название песни;
* **lyrics** - необработанный текст песни, не подходящий для лемматизации и анализа;
* **duration** - продолжительность песни (в секундах);
* **year** - год выпуска;
* **lyrics_clean** - очищенный вариант текста песни, подходящий для лемматизации;

Теперь давайте посмотрим, к каким типам данных принадлежат данные колонки:

In [ ]:
df.dtypes

* Здесь мы можем пронаблюдать, что **artist, title** и **lyrics**(а также **lyrics_clean**) принадлежат к типу **object**. Соответственно, **year** принадлежит к типу **float64**, а **duration** - к типу **int64**.

 

In [ ]:
no_artist = df['artist'].isna()
no_title = df['title'].isna()
no_lyrics = df['lyrics'].isna()
no_duration = df['duration'].isna()
no_year = df['year'].isna()

print(f"Количество строк с отсутствующим именем артиста: {no_artist.sum()}")
print(f"Количество строк с отсутствующим заголовком: {no_title.sum()}")
print(f"Количество строк с отсутствующим текстом: {no_lyrics.sum()}")
print(f"Количество строк с отсутствующей продолжительностью трека: {no_duration.sum()}")
print(f"Количество строк с отсутствующим годом выхода: {no_year.sum()}")

* Учитывая количество пропусков по датам выхода треков, мы не можем проводить анализ значения year, так как это более половины всех треков в нашей выборке. Мы не рассматриваем **lyrics_clean** в анализе данных, поскольку результат будет идентичен **lyrics**, учитывая, что **lyrics_clean** является его производной.


In [ ]:
# Загрузим модель для английского языка
nlp = spacy.load("en_core_web_sm")

# Исключаем из подборки всевозможные напевы, не являющиеся реальными словами.
custom_stop_words = ["gobble", "bidee", "ve", "ll", "oh", "ya", "ain", "don", "didn", "te",
"fasule", "tay", "ce", "la", "em", "di", "il", "er", "de", "to", "ba", "da", "doo", "zee", "boo", "mmm"]

# Добавим кастомные слова к стоп-словам
STOP_WORDS.update(custom_stop_words)

def lemmatize_text(text):
    doc = nlp(text)
    lemmas = []
    for token in doc:
        if not token.is_stop and token.is_alpha and len(token) > 1:
            lemmas.append(token.lemma_)
    return lemmas

# Применим лемматизацию к предобработанному столбцу с текстами песен
df['lemmatized'] = df['lyrics_clean'].apply(lemmatize_text)

## Анализ и визуализация

In [ ]:
# Собираем все слова из колонки 'lemmatized'
all_words = [word for row in df['lemmatized'] for word in row]

# Обрабатываем текст через spaCy
doc = nlp(" ".join(all_words))

# Создаем распределение по частям речи
pos_distribution = defaultdict(list)

for token in doc:
    pos = token.pos_  # Универсальные теги (NOUN, VERB, ADJ, ADV и т.д.)
    pos_distribution[pos].append(token.text)

# Упрощаем теги
simplified_mapping = {
    "NOUN": "noun",
    "VERB": "verb",
    "ADJ": "adj",
    "ADV": "adv",
    # Остальные категории объединяем в 'other'
}

simplified_distribution = defaultdict(list)
for pos, words in pos_distribution.items():
    simplified_pos = simplified_mapping.get(pos, "other")
    simplified_distribution[simplified_pos].extend(words)


### Топ-20 распространенных слов по их типам

In [ ]:
# Топ 20 существительных
top_noun = FreqDist(simplified_distribution['noun']).most_common(20)
print(top_noun)
# Топ 20 глаголов
top_verb = FreqDist(simplified_distribution['verb']).most_common(20)
print(top_verb)
# Топ 20 прилагательных
top_adj = FreqDist(simplified_distribution['adj']).most_common(20)
print(top_adj)
# Топ 20 наречий
top_adv = FreqDist(simplified_distribution['adv']).most_common(20)
print(top_adv)

In [ ]:
# Настройка стиля
sns.set_theme(style="whitegrid")
plt.figure(figsize=(15, 10))

# Функция для построения графиков
def plot_top_words(data, title, palette, position):
    plt.subplot(2, 2, position)
    words, counts = zip(*data)
    sns.barplot(
        x=list(counts),
        y=list(words),
        hue=list(words),
        palette=palette,
        legend=False
    )
    plt.title(title)
    plt.xlabel("Частота")
    plt.ylabel('Слова')

# Построение графиков
plot_top_words(top_noun, "Топ-20 существительных", "Blues_d", 1)
plot_top_words(top_verb, "Топ-20 глаголов", "Greens_d", 2)
plot_top_words(top_adj, "Топ-20 прилагательных", "Reds_d", 3)
plot_top_words(top_adv, "Топ-20 наречий", "Purples_d", 4)

### Характерные для исполнителей слова

**Небольшая теоретическая справка про TF-IDF анализ TF** *(Term Frequency)*:
* Относительная частота слова в документе: TF = (число вхождений слова в документе) / (общее число слов в документе).

*IDF (Inverse Document Frequency)*:
* Мера редкости слова в корпусе: IDF = log( (число документов) / (число документов, содержащих слово) ).

TF-IDF:
* TF * IDF — чем выше значение, тем важнее слово для документа в контексте всего корпуса.

In [ ]:
# Объединим леммы в строки
df['lemmatized_text'] = df['lemmatized'].apply(lambda x: ' '.join(x))

In [ ]:
# Группировка по исполнителям и объединение в одну строку
grouped = df.groupby('artist')['lemmatized_text'].agg(' '.join).reset_index()

# Создание TF-IDF матрицы
vectorizer = TfidfVectorizer(max_features=1000, stop_words = list(STOP_WORDS))   # Не забываем про расширенный набор стоп-слов
tfidf_matrix = vectorizer.fit_transform(grouped['lemmatized_text'])

# Датафрейм с весами TF-IDF
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=grouped['artist']
)

# Функция для извлечения топ-слов
def get_top_artist_words(artist, n=10):
    return tfidf_df.loc[artist].nlargest(n).to_dict()

### Визуализируем характерные слова для каждого из артистов

In [ ]:
# Настройка стиля
sns.set_theme(style="whitegrid")
plt.figure(figsize=(20, 15))  # Общий размер для всех субграфиков

# Создаем сетку 2x2 для 4 графиков
plt.subplots_adjust(hspace=0.4, wspace=0.4) # Свободное пространство между графиками

def plot_artist_tfidf(artist, palette, position, n=10):
    plt.subplot(2, 2, position)
    top_words = tfidf_df.loc[artist].nlargest(n) # количество максимально встречающихся слов

    # параметры barplot
    sns.barplot(
        x=top_words.values,           # Выцепим вес tf-idf
        y=top_words.index,            # Выцепим слово
        hue=top_words.index,
        palette=palette,
        legend=False,
        dodge=False                   # Выключаем наложение баров/столбцов
    )

    plt.title(f"Топ-{n} характерных слов для {artist}", fontsize=12)
    plt.xlabel("Вес TF-IDF", fontsize=10)
    plt.ylabel("Слова")
    plt.tick_params(axis='y', labelsize=9)

# Рисуем все графики на одной фигуре
plot_artist_tfidf('Nat “King” Cole', 'YlOrBr', 1)
plot_artist_tfidf('Frank Sinatra', 'icefire', 2)
plot_artist_tfidf('Dean Martin', 'Spectral', 3)
plot_artist_tfidf('Louis Armstrong', 'coolwarm', 4)

plt.tight_layout()
plt.show()

In [ ]:

def generate_wordcloud(artist_name=None, df=None,
                       background_color='white',
                       colormap=None,
                       colormap_list=['viridis', 'plasma', 'magma', 'cividis', 'cool', 'twilight_shifted', 'icefire'],
                       max_words=100,
                       figsize=(12, 8)):
    """
    Генерирует облако слов со случайной цветовой схемой из списка.

    Параметры:
    - artist_name: Имя исполнителя, если не указано выводится облако слов по всем исполнителям
    - colormap_list: список допустимых цветовых схем.
    - colormap: если указан, будет использован вместо случайного выбора.
    """
    # Выбор цветовой схемы
    if colormap is None:
        if colormap_list:
            colormap = random.choice(colormap_list)
        else:
            colormap = 'viridis'  # Дефолтное значение
    # Фильтрация данных
    if artist_name:
        data = df[df['artist'] == artist_name]['lemmatized_text'] # Фильтр по исполнителю
        if data.empty:
            raise ValueError(f"Исполнитель '{artist_name}' не найден в данных.")
    else:
        data = df['lemmatized_text']

    # Объединение текстов в одну строку
    all_text = ' '.join(data.astype(str))

    # Подсчет частоты слов
    word_freq = Counter(all_text.split())

    # Генерация облака
    wordcloud = WordCloud(
        width=figsize[0] * 100,                 # настроим ширину фигуры
        height=figsize[1] * 100,                # настроим высоту фигуры
        background_color=background_color,      # заливка фона
        colormap=colormap,                      # выбор цветовой палитры
        max_words=max_words                     # максимальное количество слов
    ).generate_from_frequencies(word_freq)      # сгенирируем облако на основе частот встречаемости

    # Визуализация
    plt.figure(figsize=figsize)
    plt.imshow(wordcloud, interpolation='bilinear') # отобразим облако со сглаживанием
    plt.axis('off')
    if artist_name:
        plt.title(f'Облако слов для {artist_name}', fontsize=14, pad=20)
    else:
        plt.title('Облако слов для всех исполнителей', fontsize=14, pad=20)
    plt.show()

# Пример использования для Дина Мартина
generate_wordcloud(artist_name='Frank Sinatra', df=df)
# Для всех исполнителей
generate_wordcloud(df=df)

## Выводы 

### **Анализ частоты слов**:

Чаще всего именно существительные являются центральными словами в песнях, поскольку именно существительные выступают темой и главным лейтмотивом тех или иных песен. Как ни странно и предсказуемо, именно "love" (любовь) лидирует в топ-20 слов среди существительных. Будучи одной из главных тем песен любой эпохи, в эпоху джаза любовь остается самым распространенным словом - свыше 200 упоминаний среди существительных. Это также самое распространенное слово в джазовых песнях в целом.

**Однако!** Не стоит забывать про то, что "love" - также и глагол. Пускай это и не отменяет остальных наблюдений, даже так "love" имеет вдвое больше упоминаний, чем "way", как второй глагол. Вместе с существительным вариантом, "love" является САМЫМ распространенным словом среди всех четырех топов.

Из прилагательных - стоит отметить, что почти все они, кроме двух (low, cold) имеют так или иначе положительный окрас. Наличие этих двоих слов в топе, учитывая их относительно небольшое количество (около 15 у каждого) может либо значить использование подобных слов в качестве контраста в песнях, либо то, что они встречаются лишь в небольшом количестве песен. В остальном, положительный окрас большинства прилагательных можно связать с двумя вещами: либо то, что как жанр, сам по себе джаз в основном поднимает темы с положительным окрасом, вроде любви, счастья, верности, и сами песни поэтому имеют положительный окрас. Либо это можно связать с социокультурным явлением послевоенной "золотой эпохи" и экономическим процветанием США. Закат эры свинга как музыкального жанра пришелся на первую половину сороковых, в то время как джаз захватил популярность и стал главным музыкальным жанром США как раз к второй половине сороковых и оставался таким вплоть до шестидесятых. 

В анализе глаголов стоит отметить глагол "let" - в то время, как сами поставленные нами условия это не нарушает, одной из двух пересекающихся песен, помимо "Blue Moon", является "Let it Snow" с 25 упоминаниями глагола "let". Учитывая известность обоих вариантов этих песен, как и в случае с "Blue Moon", для общего проекта это не является особо большой проблемой. В остальном глаголы и наречания не представляют особого интереса, и особо ничего не сообщают.

### **Анализ частоты слов**:

Данные столбчатые диаграммы демонстрируют частоту упоминания слов по песням каждого из отдельно взятых исполнителей на основе TF-IDF. Самым интересным фактом, пожалуй, следует отметить, что для трех из четырех исполнителей самым характерным и часто используемым остается слово "love", любовь. Не важно, используется оно в роли существительного или глагола, однако встречается оно значительно чаще других.

И только Луи Армстронг предпочитает любви вкусные чизкейки - вес этого слова у него выше 0.40, что, по меркам IDF, весьма много. Единственный, у кого главное слово имеет схожий вес - Нэт Кинг Коул. В частности, можно заметить, что вес у "love" в его случае также заметно выше 0.40 и ближе к 0.50, однако тут следует учитывать саму формулу TF-IDF, а также то, что в его случае выборка в почти три раза меньше, чем у остальных музыкантов, и анализ не может считаться полностью правдоподобным и корректным.

Отдельно также выделяется Дин Мартин - в топе его характерных слов встречаются не только напевы "bon" и "boom", но и иностранное "si", присутсвовавшее в двух песнях, "C'est si bon", и "Mambo Italliano".

## Обсуждение

**Что мы хотели сделать в рамках нашего исследования и что сделать удалось**

Изначально нашим планом было проанализировать лишь три исполнителя, однако в процессе мы решили добавить четвертого (Луи Армстронга) для более широкой выборки. Это также позволило нам компенсировать тот факт, что у Нэта Кинг Коула лишь шестнадцать песен вместо пятидесяти, как у всех остальных. Можно считать, что самую основную часть работы (выяснение продолжительности песен, анализ наиболее часто встречаемых слов) мы выполнили на "отлично".
 
**Что не удалось сделать и почему**

Одной из главных визуализаций данных, которую мы планировали вставить, изначально был анализ наиболее значимых для этих музыкантов лет через подсчет песен по датам их выпуска и создания на этой основе графика. Проблема возникла с музыкальной базой данных и её api - из 166 треков 90 оказалось без дат. Следовательно, анализ данных получился бы по меньшей мере практически полностью недостоверным.  Мы также были вынуждены отказаться от полноценной визуализации с wordcloud в связи с некоторыми багами, из-за которых он мог цеплять целые словосочетания вместо одного слова. В один момент мы также рассматривали идею убрать дубликаты песен из подборки, однако в конечном счете решили, что это не имеет смысла относительно того, что именно мы анализируем. 

**Как наше исследование можно было бы улучшить**

Возможно, расширить выборку ещё на несколько знаменитых джазовых исполнителей: возможно, Пегги Ли, Бинга Кросби, Джона Колтрэйна, Билли Холидея, Сэмми Дэвиса. Если бы мы расширили выборку, из нишевого проекта это стало бы более значимой работой. Кроме того, вместо 50 наиболее знаменитых песен каждого музыканта мы могли взять всю их дискографию. Однако в таком случае также пришлось бы чистить подборку от возможных дубликатов некоторых песен, вроде Blue Moon. Из более экспериментальных идей - попробовать пошарить по американским сайтам в поисках данных о продажах копий джазовых альбомов Фрэнка Синатры, как одного из самых известных исполнителей джазовых песен (даже если и не джазмена в классическом понимании термина) и сопоставить с продажами рок-н-ролльных альбомов и синглов Элвиса Пресли, как самого известного исполнителя в жанре рок-н-ролла, дабы отследить момент, где джаз уступил рок-н-роллу в популярности среди простых обывателей.

**Fun Fact:**
В нашем готовом датасете (пусть и сделанном на основе скрипта) песня "Blue Moon" встречается два раза. Изначально не написанная ни Синатрой, ни Дином Мартином, она была создана в тридцатых и исполнялась огромным количеством певцов и групп в самые разные времена. Её исполнял даже Элвис Пресли в самой начале своей карьеры, причем его звучание песни сильно отличается не только от более торжественных версий, но и от его более поздних работ.

**Кому наше исследование может быть полезно (или что можно сделать дополнительно, чтобы оно было полезным)**

Если провести изменения, которые мы описали в прошлом блоке по поводу улучшения нашего проекта, чисто теоретически, оно бы могло использоваться в полноценной научной работе по поводу джаза. Провести подобное с рок-н-роллом в тех же годах, а также с ещё парой доминирующих жанров, вроде диско и более классического рока/рэпа, и подобную таблицу можно было бы использовать в научной работе по поводу различных музыкальных эпох в США, и как одни жанры выходили из моды и заменялись другими с развитием культуры. В зависимости от типа исследования также может иметь смысл удаление дубликатов Blue Moon и Let it Snow.

В общем и целом, это исследование так и останется весьма специфичным и нишевым исследованием, однако базовая логика может быть улучшена и адаптирована и для более серьезных работ.